In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

In [ ]:
players = pd.read_csv("../input/nfl-big-data-bowl-2022/players.csv")
plays = pd.read_csv("../input/nfl-big-data-bowl-2022/plays.csv")
games = pd.read_csv("../input/nfl-big-data-bowl-2022/games.csv")
scouting = pd.read_csv("../input/nfl-big-data-bowl-2022/PFFScoutingData.csv")
tracking2018 = pd.read_csv("../input/nfl-big-data-bowl-2022/tracking2018.csv",nrows=5000000)
#tracking2019 = pd.read_csv("tracking2019.csv")
#tracking2020 = pd.read_csv("tracking2020.csv")

# Plays Table 

In [ ]:
plays

In [ ]:
pd.options.display.max_rows = 10
# plays_on_game = plays[(plays['gameId']==2018102101)]# & (plays['specialTeamsPlayType']=="Punt")]
plays_reduced = plays[["gameId","playId","kickerId","possessionTeam","specialTeamsPlayType","specialTeamsResult",
              "kickLength","kickReturnYardage","playResult"]]
plays_reduced.reset_index(inplace=True)
plays_reduced = plays_reduced.drop("index",axis=1)
plays_reduced

# Scouting Table

In [ ]:
scouting

In [ ]:
scouting_reduced = scouting[['gameId','playId','kickType','kickDirectionIntended','kickDirectionActual','kickoffReturnFormation']]

In [ ]:
scouting_reduced

## Scouting and Plays tables joined

In [ ]:
joined_tables = pd.concat([scouting_reduced,plays_reduced],axis = 1,join="inner")
joined_tables = joined_tables.loc[:,~joined_tables.columns.duplicated()]

In [ ]:
joined_tables

# Average KickLength per Player 

In [ ]:
unique_kickers = plays['kickerId'].unique()

In [ ]:
unique_kickers

In [ ]:
len(unique_kickers)

In [ ]:
players_avg = pd.DataFrame()
players_avg = plays_reduced[["gameId","playId","kickerId"]]
players_avg['Avg_Kick_Length'] = plays.groupby(['kickerId'])['kickLength'].transform('mean')

In [ ]:
players_avg

In [ ]:
len(players_avg["Avg_Kick_Length"].unique())

# Scouting + Plays + KickLengthAvg 

In [ ]:
stats_table = pd.concat([joined_tables,players_avg],axis = 1,join="inner")
stats_table = stats_table.loc[:,~stats_table.columns.duplicated()]

In [ ]:
stats_table

In [ ]:
stats_table.columns

In [ ]:
stats_table = stats_table[['gameId', 'playId', 'kickerId','kickDirectionActual',
       'specialTeamsPlayType', 'specialTeamsResult',
       'kickLength', 'Avg_Kick_Length' ,'playResult']]

In [ ]:
stats_table

In [ ]:
avg_playresult = (stats_table["playResult"].sum())/len(stats_table)

In [ ]:
avg_playresult

In [ ]:
def net_result(df,avg):
    if (df["playResult"]>= avg): return 1
    else: return 0       

#---------------------------------------------
#Left: 1; Center: 0; Right: 2
def left_right(df):
    if (df["kickDirectionActual"] == "L"):
        return 1
    elif (df["kickDirectionActual"] == "C"):
        return 0
    else:
        return 2

In [ ]:
stats_table["Target"] = stats_table.apply(lambda x: net_result(x,avg_playresult),axis=1)
stats_table["KickDirection"] = stats_table.apply(lambda x: left_right(x),axis=1)

In [ ]:
stats_table.columns

In [ ]:
stats_table = stats_table[['gameId', 'playId', 'kickerId','specialTeamsPlayType','specialTeamsResult',
                          'kickDirectionActual',
                           'kickLength','KickDirection','Avg_Kick_Length','playResult']]

# Final Tables (only punts / kickoffs)

In [ ]:
punt_table = stats_table[stats_table["specialTeamsPlayType"] == "Punt"]
kickoff_table = stats_table[stats_table["specialTeamsPlayType"] == "Kickoff"]

In [ ]:
avg_kickoffresult = int((kickoff_table["playResult"].sum())/len(kickoff_table))
avg_puntresult = int((punt_table["playResult"].sum())/len(punt_table))

In [ ]:
avg_kickoffresult, avg_puntresult

In [ ]:
kickoff_table["Diff_vs_Avg"] = kickoff_table['playResult']-avg_kickoffresult
punt_table["Diff_vs_Avg"] = punt_table['playResult']-avg_puntresult

In [ ]:
#Redo the Target class for each individual table since the original "Target" class is mixed with values from both
#Punts and Kickoffs

In [ ]:
kickoff_table["Target"] = kickoff_table.apply(lambda x: net_result(x,avg_kickoffresult),axis=1)
punt_table["Target"] = punt_table.apply(lambda x: net_result(x,avg_puntresult),axis=1)

In [ ]:
pd.set_option("display.max_rows", 11)
kickoff_table.isnull().sum()

In [ ]:
kickoff_table.dropna(axis=0, inplace = True)

In [ ]:
punt_table.isnull().sum()

In [ ]:
punt_table.dropna(axis=0,inplace = True)

# Punt Analysis

In [ ]:
punt_table.columns

In [ ]:
punt_table = punt_table[['gameId', 'playId', 'kickerId', 'specialTeamsPlayType',
       'specialTeamsResult', 'kickDirectionActual', 'kickLength',
       'KickDirection', 'Avg_Kick_Length','Diff_vs_Avg','playResult',
       'Target']]

In [ ]:
#Drop the PlayResult to only have the Target with 1s and 0s. (or maybe use the play Result table)
#punt_table.drop(['playResult'],axis=1,inplace=True)

In [ ]:
punt_table.head()

In [ ]:
#For punts table:

#FOR TARGETS:
#USE:
#-1 for Target column
#-2 for Diff_vs_Avg column (which is playResult - Avg_kick_length of all players in all plays)
#-3 for playResult Column

inputs_punts = punt_table[punt_table.columns[6:9]].to_numpy()  #In here include up to number 10 if you want to try "Diff_vs_Avg" as an input
targets_punts = punt_table[punt_table.columns[-1]].to_numpy()
inputs_punts, targets_punts
#dont understand why (targets_punts) is a 2-d array lol; that's why I use "targets_punts[:,0]"

In [ ]:
#For PUNTS
#Dividing the dataframe into a train and a test sections
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(inputs_punts,targets_punts,test_size=0.2, random_state = 42)

In [ ]:
x_test,y_test

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(x_train, y_train)

## predict!
y_train_pred = DT_model.predict(x_train)
y_test_pred = DT_model.predict(x_test)

## so we get the pred result
y_test_pred[:100]

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
# function for visualizing confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix',
                          cmap=sns.cubehelix_palette(as_cmap=True)):
    """
    This function is modified from: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    classes.sort()
    tick_marks = np.arange(len(classes))    
    
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels = classes,
           yticklabels = classes,
           title = title,
           xlabel = 'True label',
           ylabel = 'Predicted label')

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    ylim_top = len(classes) - 0.5
    plt.ylim([ylim_top, -.5])
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, CategoricalNB 
naive_classifier = CategoricalNB()
naive_classifier.fit(x_train, y_train)

In [ ]:
# short DataFrame of visualizations of how many correct and incorrect the classifier got:

# make the predicitions with the test data
y_train_pred = naive_classifier.predict(x_train)

# establish how many correct, incorrect predictions, and accuracy (train):
correct_predictions_train = (y_train == y_train_pred).sum()
incorrect_predictions_train = y_train.size - correct_predictions_train
train_accuracy = round((naive_classifier.score(x_train,y_train))*100,2)

results_naive = correct_predictions_train, incorrect_predictions_train, train_accuracy

df_results = pd.DataFrame([results_naive], columns = ['Correct','Incorrect', 'Accuracy %'])
df_results

In [ ]:
y_test_pred = naive_classifier.predict(x_test)

# establish how many correct, incorrect predictions, and accuracy (test):
correct_predictions = (y_test == y_test_pred).sum()
incorrect_predictions = y_test.size - correct_predictions
accuracy = round((naive_classifier.score(x_test,y_test))*100,2)

results_naive = correct_predictions, incorrect_predictions, accuracy

df_results = pd.DataFrame([results_naive], columns = ['Correct','Incorrect', 'Accuracy %'])
df_results

In [ ]:
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# build Logistic Regression
LR_model = LogisticRegression(solver='liblinear', C=10, n_jobs=-1, max_iter=1000, tol=0.0001, verbose=2)

# training!
LR_model.fit(x_train,y_train)

# predict!
y_train_pred = LR_model.predict(x_train)
y_test_pred = LR_model.predict(x_test)

In [ ]:
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

In [ ]:
# precision, recall, f1-score,
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred=y_test_pred))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier(n_estimators=1000)

clf.fit(x_train,y_train)

y_train_pred=clf.predict(x_train)
y_test_pred=clf.predict(x_test)

In [ ]:
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# K-nearest Neighbors

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics

In [ ]:
sc= StandardScaler()

sc.fit(x_train)
x_train= sc.transform(x_train)

sc.fit(x_test)
x_test= sc.transform(x_test)

inputs_punts.shape

In [ ]:
error1= []
error2= []
for k in range(1,15):
    knn= KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train,y_train)
    y_pred1= knn.predict(x_train)
    error1.append(np.mean(y_train!= y_pred1))
    y_pred2= knn.predict(x_test)
    error2.append(np.mean(y_test!= y_pred2))
plt.figure(figsize=(15,10))
plt.plot(range(1,15),error1,label="train")
plt.plot(range(1,15),error2,label="test")
plt.xlabel('k Value')
plt.ylabel('Error')
plt.legend()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=13)
 
knn.fit(x_train, y_train)
 
print(knn.predict(x_train))
print(knn.predict(x_test))

In [ ]:
# Calculate the accuracy of the model
print(knn.score(x_train, y_train))
print(knn.score(x_test, y_test))

In [ ]:
# check by confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Kick Analysis

In [ ]:
kickoff_table.columns

In [ ]:
kickoff_table = kickoff_table[['gameId', 'playId', 'kickerId', 'specialTeamsPlayType',
       'specialTeamsResult', 'kickDirectionActual', 'kickLength',
       'KickDirection', 'Avg_Kick_Length','Diff_vs_Avg','playResult',
       'Target']]

In [ ]:
kickoff_table.head()

In [ ]:
#For kickoffs table:

#FOR TARGETS:
#USE:
#-1 for Target column
#-2 for Diff_vs_Avg column (which is playResult - Avg_kick_length of all players in all plays)
#-3 for playResult Column

inputs_kickoff = kickoff_table[punt_table.columns[6:9]].to_numpy()  #In here include up to number 10 if you want to try "Diff_vs_Avg" as an input
targets_kickoff = kickoff_table[punt_table.columns[-1]].to_numpy()
inputs_kickoff, targets_kickoff
#dont understand why (targets_punts) is a 2-d array lol; that's why I use "targets_punts[:,0]"

In [ ]:
#For KICKOFFS
#Dividing the dataframe into a train and a test sections
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(inputs_kickoff,targets_kickoff,test_size=0.2, random_state = 42)

In [ ]:
x_test,y_test

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(x_train, y_train)

## predict!
y_train_pred = DT_model.predict(x_train)
y_test_pred = DT_model.predict(x_test)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Naive Bayes

In [ ]:
naive_classifier = CategoricalNB()
naive_classifier.fit(x_train, y_train)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Logistic Regression

In [ ]:
# build Logistic Regression
LR_model = LogisticRegression(solver='liblinear', C=10, n_jobs=-1, max_iter=1000, tol=0.0001, verbose=2)

# training!
LR_model.fit(x_train,y_train)

# predict!
y_train_pred = LR_model.predict(x_train)
y_test_pred = LR_model.predict(x_test)

In [ ]:
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))
print('testing accuracy: {}'.format(round(acc_test, 2)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')

# Random Forest

In [ ]:
clf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

y_train_pred=clf.predict(x_train)
y_test_pred=clf.predict(x_test)

In [ ]:
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

print('training accuracy: {}'.format(round(acc_train, 4)))
print('testing accuracy: {}'.format(round(acc_test, 4)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')


# K-Nearest Neighbors

In [ ]:
sc= StandardScaler()

sc.fit(x_train)
x_train= sc.transform(x_train)

sc.fit(x_test)
x_test= sc.transform(x_test)

inputs_punts.shape

In [ ]:
error1= []
error2= []
for k in range(1,15):
    knn= KNeighborsClassifier(n_neighbors=k)
    knn.fit(x_train,y_train)
    y_pred1= knn.predict(x_train)
    error1.append(np.mean(y_train!= y_pred1))
    y_pred2= knn.predict(x_test)
    error2.append(np.mean(y_test!= y_pred2))
plt.figure(figsize=(15,10))
plt.plot(range(1,15),error1,label="train")
plt.plot(range(1,15),error2,label="test")
plt.xlabel('k Value')
plt.ylabel('Error')
plt.legend()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=11)
 
knn.fit(x_train, y_train)
 
print(knn.predict(x_train))
print(knn.predict(x_test))

In [ ]:
# Calculate the accuracy of the model
print(knn.score(x_train, y_train))
print(knn.score(x_test, y_test))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_test_pred) 
print(cm)

my_tags = ['0','1']
plot_confusion_matrix(cm, classes=my_tags, title='Confusion matrix')